In [ ]:
from itertools import groupby
from pathlib import Path
from time import time

import numpy as np
import pandas as pd
import sqlalchemy as sa


from faker import Faker
from tqdm.notebook import tqdm

In [ ]:
db_file = Path("db.sqlite")
db_file.unlink(missing_ok=True)

In [ ]:
constr = f"sqlite:///{db_file}"
engine = sa.create_engine(constr, echo=True)
metadata_obj = sa.MetaData()

In [ ]:
user = sa.Table(
    "user",
    metadata_obj,
    sa.Column("user_id", sa.Integer, primary_key=True),
    sa.Column("user_name", sa.String(16), nullable=False),
    sa.Column("email_address", sa.String(60)),
    sa.Column("height", sa.Float),
    sa.Column("blood_type", sa.String(2)),
)

In [ ]:
metadata_obj.create_all(engine)

In [ ]:
ins = user.insert(values=[(1, "someone", "e@b.com", 123.1, "AB")])

In [ ]:
engine.execute(ins)

In [ ]:
pd.read_sql("SELECT * FROM user", con=constr)

In [ ]:
del_stmt = user.delete().where(user.c.user_name=="someone")

In [ ]:
engine.execute(del_stmt)

In [ ]:
pd.read_sql("SELECT * FROM user", con=constr)

In [ ]:
ff = Faker()
rng = np.random.RandomState(742)

In [ ]:
# faker on its own is very slow
name_selection = [ff.name() for _ in range(3000)]
email_selection = [ff.email() for _ in range(3000)]

In [ ]:
exp = 6
n = 10 ** exp
user_df = pd.DataFrame(
    {
        0: range(n),
        1: rng.choice(name_selection, n),
        2: rng.choice(email_selection, n),
        3: rng.normal(165, 10, n),
        4: rng.choice(["A", "B", "AB", "0"], n),
    }
)

In [ ]:
user_df

In [ ]:
value_list = [*user_df.itertuples(name=None, index=False)]

In [ ]:
value_list[:3]

In [ ]:
engine.echo = False

In [ ]:
for gid, value_group in tqdm(groupby(value_list, lambda t: (t[0] // 5000))):
    ins = user.insert(values=[*value_group])
    engine.execute(ins)

In [ ]:
%%time
pd.read_sql("SELECT * FROM user", con=constr)

In [ ]:
(user_df.to_numpy() == pd.read_sql("SELECT * FROM user", con=constr).to_numpy()).all()

In [ ]:
%%time
pd.read_sql("SELECT * FROM user WHERE height > 160 AND blood_type != 'B'", con=constr)

In [ ]:
db2_file = Path("db2.sqlite")
db2_file.unlink(missing_ok=True)

In [ ]:
constr2 = f"sqlite:///{db2_file}"
engine2 = sa.create_engine(constr2)
metadata_obj2 = sa.MetaData()

In [ ]:
user_indexed = sa.Table(
    "user",
    metadata_obj2,
    sa.Column("user_id", sa.Integer, primary_key=True),
    sa.Column("user_name", sa.String(16), nullable=False),
    sa.Column("email_address", sa.String(60)),
    sa.Column("height", sa.Float),
    sa.Column("blood_type", sa.String(2)),
    sa.Index("findex", "height", "blood_type")
)

In [ ]:
metadata_obj2.create_all(engine2)

In [ ]:
for gid, value_group in tqdm(groupby(value_list, lambda t: (t[0] // 5000))):
    ins = user_indexed.insert(values=[*value_group])
    engine2.execute(ins)

In [ ]:
%%time
pd.read_sql("SELECT * FROM user", con=constr2)

In [ ]:
%%time
pd.read_sql("SELECT * FROM user WHERE height > 160 AND blood_type != 'B'", con=constr2)

In [ ]:
filters = [
    "height > 160 AND blood_type < 'B'",
    "height > 180 AND blood_type == 'A'",
    "height < 140 AND blood_type < 'A'",
    "height > 175 AND blood_type != 'A' AND height < 183 AND blood_type != 'AB'",
]

recs = []
for _filt in filters:

    stmt = f"SELECT * FROM user WHERE {_filt}"

    rec = {"filstr": _filt}

    for _con, name in [(constr, "regular"), (constr2, "indexed")]:
        stime = time()
        _df = pd.read_sql(stmt, con=_con)
        rec[name] = time() - stime
    rec["size"] = _df.shape[0]

    recs.append(rec)

In [ ]:
pd.DataFrame(recs).assign(rate=lambda df: df["regular"] / df["indexed"])

In [ ]:
# in MB
db_file.stat().st_size / 2 ** 20

In [ ]:
db2_file.stat().st_size / 2 ** 20